# Средняя продолжительность формального образования для лиц в возрасте 15–64 лет

In [147]:
import pandas as pd
import numpy as np 

from helpers import (
    get_data_with_full_locations_and_years, 
    get_omissions, 
    print_difference_of_sets,
)

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте "Наш мир в данных" по ссылке

https://ourworldindata.org/grapher/mean-years-of-schooling-long-run

> **Источники данных:**
> [Барро и Ли (2015)](http://www.barrolee.com/);
> [Ли и Ли (2016)](https://barrolee.github.io/BarroLeeDataSet/DataLeeLee.html) - 
> с обработкой [Our World In Data](https://ourworldindata.org/grapher/mean-years-of-schooling-long-run). \
> **Руководство по цитированию:** Пожалуйста, укажите все источники, перечисленные выше. \
> Данные, предоставленные  сторонними источниками через Our World in Data, \
> остаются подпадающими под условия лицензии исходных поставщиков.

In [148]:
# Загрузим таблицу c данными об образовании
data = pd.read_csv('../data/ourworldindata_schooling.csv')

data.head()

,Entity,Code,Year,Combined - average years of education for 15-64 years male and female youth and adults
0,Afghanistan,AFG,2000,3.03
1,Afghanistan,AFG,2005,3.32
2,Afghanistan,AFG,2010,3.93
3,Afghanistan,AFG,2015,4.81
4,Afghanistan,AFG,2020,5.69


Описание полей:

- `Entity` - название страны
- `Code` - код страны
- `Year` - год
- `Combined - average years of education for 15-64 years male and female youth and adults` - средняя продолжительность обучения для мужчин и женщин в возрасте 15-64 лет, молодых и взрослых

In [149]:
# Загрузим также таблицу с данными стран и регионов
regions_and_locations_data = pd.read_csv(
    '../data/regions_and_locations.csv'
)

regions_and_locations_data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location
0,AFR,Africa,AGO,Angola
1,AFR,Africa,BDI,Burundi
2,AFR,Africa,BEN,Benin
3,AFR,Africa,BFA,Burkina Faso
4,AFR,Africa,BWA,Botswana


In [150]:
# Посмотрим, за какие годы представлены данные
data['Year'].unique()

array([2000, 2005, 2010, 2015, 2020], dtype=int64)

Данные представлены с промежутком в 5 лет, это нужно будет учесть.

In [151]:
# Переименуем последний столбец
data.rename(
    columns={
        'Combined - average years of education for 15-64 years male and female youth and adults': 'Schooling',
    },
    inplace=True,
)

# Посмотрим на информацию о столбцах
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Entity     660 non-null    object 
 1   Code       630 non-null    object 
 2   Year       660 non-null    int64  
 3   Schooling  660 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 20.8+ KB


Типы столбцов соответствуют содержимому.\
В столбце с кодами стран `Code` есть пропуски, позже разберемся с ними.

In [152]:
# Посмотрим на соответствия стран в таблицах
print_difference_of_sets(
    regions_and_locations_data['SpatialDimValueCode'].unique(),
    data['Code'].unique()
)

Элементы, которые есть в первом множестве, но отсутствуют во втором: 
 {'DJI', 'GIN', 'SOM', 'KIR', 'SYC', 'MNE', 'BIH', 'GEO', 'BHS', 'COM', 'MKD', 'BFA', 'ERI', 'SLB', 'AGO', 'ETH', 'LBN', 'GRD', 'GNB', 'SUR', 'TLS', 'TKM', 'WSM', 'ATG', 'VUT', 'GNQ', 'AZE', 'BTN', 'STP', 'VCT', 'NGA', 'UZB', 'CPV', 'MDG', 'OMN', 'LCA', 'BLR', 'FSM', 'TCD'}

Элементы, которые есть во втором множестве, но отсутствуют в первом: 
 {nan, 'TWN', 'OWID_WRL', 'HKG', 'REU', 'MAC'}



Данные по отсутсвующим странам заполним средним по региону.

А пока оставим в таблице `data` только те страны, которые есть в `regions_and_locations_data`.

In [153]:
# Коды рассматриваемых стран
location_codes = list(regions_and_locations_data['SpatialDimValueCode'].unique())

# Маска принадлежности страны к заданному множеству
mask = data['Code'].isin(location_codes)
# Удалим записи, которые не подходят под маску
data.drop(data[~mask].index, inplace=True)

# Посмотрим, остались ли пропуски
data.isna().sum()

Entity       0
Code         0
Year         0
Schooling    0
dtype: int64

Пропусков больше нет.

In [154]:
# Уберем поле с названием страны
data.drop(columns=['Entity'], inplace=True)

# Переименуем столбцы
data.rename(
    columns={
        'Code': 'SpatialDimValueCode',
        'Year': 'Period',
    },
    inplace=True,
)

data.head()

,SpatialDimValueCode,Period,Schooling
0,AFG,2000,3.03
1,AFG,2005,3.32
2,AFG,2010,3.93
3,AFG,2015,4.81
4,AFG,2020,5.69


Добавим в таблицу столбец с кодом региона.

In [155]:
# Таблица с кодами стран и регионов
regions_codes = regions_and_locations_data[['ParentLocationCode', 'SpatialDimValueCode']]

# Смержим ее с основной
data = data.merge(
    regions_codes,
    on='SpatialDimValueCode',
    how='left',
)

# Переставим столбцы
data = data.reindex(
    columns=['ParentLocationCode', 'SpatialDimValueCode', 'Period', 'Schooling'],
)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
0,EMR,AFG,2000,3.03
1,EMR,AFG,2005,3.32
2,EMR,AFG,2010,3.93
3,EMR,AFG,2015,4.81
4,EMR,AFG,2020,5.69


In [156]:
# Убедимся, что нет пропусков
data.isna().sum().sum()

0

In [157]:
# Посмотрим, как распределены данные по годам
data['Period'].value_counts()

Period
2015    142
2020    142
2000    108
2005    108
2010    108
Name: count, dtype: int64

Данные по странам представлены не за все года, добавим строки по отсутствующм годам, чтобы явно увидеть пропуски.

In [158]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

data.isna().sum()

ParentLocationCode       0
SpatialDimValueCode      0
Period                   0
Schooling              102
dtype: int64

Исследуем эти пропуски и по итогу решим, как их заполнить.

In [159]:
# Сформируем таблицу с пропущенными строками
omission_data = get_omissions(data)

omission_data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
4,EUR,ARM,2000,NaN
7,EMR,BHR,2000,NaN
14,AFR,BWA,2000,NaN
16,WPR,BRN,2000,NaN
18,AFR,BDI,2000,NaN


In [160]:
# Посмотрим, за какие года нет данны
omission_data['Period'].value_counts()

Period
2000    34
2005    34
2010    34
Name: count, dtype: int64

То есть, 2015 и 2020 у всех стран заполнены.\
Не заполнены только начальные года.\
И за каждый год не заполнено одинаковое количество стран.

Отсортируем данные в исходной таблице так, чтобы первыми шли года с заполненными значениями `Schooling`.\
И заполним пропуски с помощью интерполяции.

In [161]:
# Выполним сотрировку
data = data.sort_values(by=['SpatialDimValueCode', 'Period'], ascending=False)
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)
data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
0,AFR,ZWE,2020,8.32
1,AFR,ZWE,2015,8.03
2,AFR,ZWE,2010,7.86
3,AFR,ZWE,2005,7.65
4,AFR,ZWE,2000,7.26


In [162]:
# Заполним пропуски с помощью интерполяции
data = data.groupby(by=['SpatialDimValueCode']).apply(
    lambda group: group.interpolate(method='index')
)

# Убедимся, что нет пропусков
data.isna().sum()

ParentLocationCode     0
SpatialDimValueCode    0
Period                 0
Schooling              0
dtype: int64

Теперь выполним сортировку, чтобы года шли по возрастанию.

In [163]:
data = data.reset_index(drop=True)
data = data.sort_values(by=['SpatialDimValueCode', 'Period'])
data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
4,EMR,AFG,2000,3.03
3,EMR,AFG,2005,3.32
2,EMR,AFG,2010,3.93
1,EMR,AFG,2015,4.81
0,EMR,AFG,2020,5.69


В основной таблице, которая формируется в файле [01_main.ipynb](./01_main.ipynb),\
мы рассматриваем данные за каждый год с 2000 по 2020.\
Добавим в данную таблицу строки со всеми годами.

In [164]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    },
    1
)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
0,EMR,AFG,2000,3.03
1,EUR,ALB,2000,9.93
2,EMR,ARE,2000,9.35
3,AMR,ARG,2000,9.08
4,EUR,ARM,2000,10.45


In [165]:
# Посмотрим на количество пропусков
data.isna().sum()

ParentLocationCode        0
SpatialDimValueCode       0
Period                    0
Schooling              2272
dtype: int64

Заполним пропуски за отстутствующие года с помощью интерполяции.

In [166]:
# Для этого снова отсортируем данные таблицы
data = data.sort_values(by=['SpatialDimValueCode', 'Period'])
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)
data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
0,EMR,AFG,2000,3.03
1,EMR,AFG,2001,NaN
2,EMR,AFG,2002,NaN
3,EMR,AFG,2003,NaN
4,EMR,AFG,2004,NaN


In [167]:
# Заполним пропуски с помощью интерполяции
data = data.groupby(by=['SpatialDimValueCode']).apply(
    lambda group: group.interpolate(method='index')
)

data.head()

ParentLocationCode SpatialDimValueCode Period  Schooling
SpatialDimValueCode                                                           
AFG                 0                EMR                 AFG   2000      3.030
                    1                EMR                 AFG   2001      3.088
                    2                EMR                 AFG   2002      3.146
                    3                EMR                 AFG   2003      3.204
                    4                EMR                 AFG   2004      3.262

In [168]:
# Убедимся, что пропусков нет
data.isna().sum().sum()

0

Теперь добавим отсутствующие страны и заполним данные по ним средним по региону.

In [169]:
# Соберем список с кодами отсутствующих стран
mising_location_codes = \
    set(regions_and_locations_data['SpatialDimValueCode'].unique()) - \
    set(data['SpatialDimValueCode'].unique())
    
mising_location_codes = list(mising_location_codes)

# Маска принадлежности кода страны к списку отсутствующих стран
mask = regions_and_locations_data['SpatialDimValueCode'].isin(mising_location_codes)
# Отфильтруем данные отсутствующих стран
mising_locations_data = regions_and_locations_data[mask]
# Оставим в данных только коды региона и страны
mising_locations_data = mising_locations_data[['ParentLocationCode', 'SpatialDimValueCode']]
# Поле с годом заполним начальным 2000-м
mising_locations_data['Period'] = 2000
# Значение поля с длительностью обучения мы пока не знаем, заполним его NaN
mising_locations_data['Schooling'] = np.nan

# Посмотрим на результат
mising_locations_data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Schooling
0,AFR,AGO,2000,NaN
3,AFR,BFA,2000,NaN
10,AFR,COM,2000,NaN
11,AFR,CPV,2000,NaN
13,AFR,ERI,2000,NaN


In [170]:
# Присоединим его к основной таблице
data = pd.concat([
    data,
    mising_locations_data,
], ignore_index=True)

# Для добавленных стран заполним строки с годами с 2001 по 2020
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    },
)

# Заполним пропуски средним по региону
data['Schooling'] = \
    data.groupby(['ParentLocationCode', 'Period'])['Schooling'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [171]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [172]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/ourworldindata_schooling_prepared.csv', index=False)